# Getting Started with Fine-Tuning Mistral 7B

This notebook shows you a simple example of how to LoRA finetune Mistral 7B. You can run this notebook in Google Colab with Pro + account with A100 and 40GB RAM.

<a target="_blank" href="https://colab.research.google.com/github/smartrics/mistral-finetune/blob/main/tutorials/mistral_finetune_7b.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


Check out `mistral-finetune` Github repo to learn more: https://github.com/smartrics/mistral-finetune/

## Installation

Clone the `mistral-finetune` repo:


In [3]:
import os
import subprocess

repo_dir = "/content/mistral-finetune"
repo_url = "https://github.com/smartrics/mistral-finetune.git"

if os.path.isdir(repo_dir):
    print("Directory 'mistral-finetune' exists. Pulling latest changes...")
    subprocess.run(["git", "-C", repo_dir, "pull"], check=True)
else:
    print("Directory 'mistral-finetune' does not exist. Cloning repository...")
    subprocess.run(["git", "clone", repo_url, repo_dir], check=True)
print("finished!")

Directory 'mistral-finetune' exists. Pulling latest changes...
finished!


Install all required dependencies:

In [4]:
!pip install -r /content/mistral-finetune/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of xformers to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━

## Model download

In [5]:
!pip install huggingface_hub

In [6]:
# huggingface login
from huggingface_hub import notebook_login

notebook_login()

In [7]:
from huggingface_hub import snapshot_download
from pathlib import Path

mistral_models_path = Path.home().joinpath('mistral_models', '7B-Instruct-v0.3')
mistral_models_path.mkdir(parents=True, exist_ok=True)

snapshot_download(repo_id="mistralai/Mistral-7B-Instruct-v0.3", allow_patterns=["params.json", "consolidated.safetensors", "tokenizer.model.v3"], local_dir=mistral_models_path)

! cp -r /root/mistral_models/7B-Instruct-v0.3 /content/mistral_models
! rm -r /root/mistral_models/7B-Instruct-v0.3

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

consolidated.safetensors:   0%|          | 0.00/14.5G [00:00<?, ?B/s]

tokenizer.model.v3:   0%|          | 0.00/587k [00:00<?, ?B/s]

params.json:   0%|          | 0.00/202 [00:00<?, ?B/s]

## Dataset

Use the data in `/content/mistral-finetune/data`

In [8]:
!ls /content/mistral-finetune/data

prepare.py  test_data.jsonl  training_data.jsonl  validation_data.jsonl


In [9]:
# navigate to the mistral-finetune directory
%cd /content/mistral-finetune/

/content/mistral-finetune


In [11]:
# Now you can verify your training yaml to make sure the data is correctly formatted and to get an estimate of your training time.

!python -m utils.validate_data --train_yaml example/7B.yaml


0it [00:00, ?it/s]Validating data/test_data.jsonl ...

  0% 0/50 [00:00<?, ?it/s]
 10% 5/50 [00:00<00:01, 43.09it/s]
 20% 10/50 [00:00<00:00, 43.93it/s]
 30% 15/50 [00:00<00:00, 44.50it/s]
 40% 20/50 [00:00<00:00, 44.50it/s]
 50% 25/50 [00:00<00:00, 44.82it/s]
 60% 30/50 [00:00<00:00, 44.46it/s]
 70% 35/50 [00:00<00:00, 44.58it/s]
 80% 40/50 [00:00<00:00, 44.53it/s]
 90% 45/50 [00:01<00:00, 44.55it/s]
100% 50/50 [00:01<00:00, 44.50it/s]
1it [00:01,  1.13s/it]
No errors! Data is correctly formatted!
Stats for data/test_data.jsonl 
 -------------------- 
 {
    "expected": {
        "eta": "00:33:38",
        "data_tokens": 477199,
        "train_tokens": 78643200,
        "epochs": "164.80",
        "max_steps": 300,
        "data_tokens_per_dataset": {
            "data/test_data.jsonl": "477199.0"
        },
        "train_tokens_per_dataset": {
            "data/test_data.jsonl": "78643200.0"
        },
        "epochs_per_dataset": {
            "data/test_data.jsonl": "164.8"
     

## Start training

In [12]:
# these info is needed for training
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [29]:
# define training configuration
# for your own use cases, you might want to change the data paths, model path, run_dir, and other hyperparameters

config = """
data:
  instruct_data: "data/test_data.jsonl"  # Fill
  data: ""  # Optionally fill with pretraining data
  eval_instruct_data: "data/validation_data.jsonl"  # Optionally fill

# model
model_id_or_path: "/content/mistral_models"  # Change to downloaded path
lora:
  rank: 64

# optim
seq_len: 16384
batch_size: 1
max_steps: 500
optim:
  lr: 6.e-5
  weight_decay: 0.1
  pct_start: 0.05

# other
seed: 0
log_freq: 1
eval_freq: 100
no_eval: False
ckpt_freq: 100

save_adapters: True  # save only trained LoRA adapters. Set to `False` to merge LoRA adapter into the base model and save full fine-tuned model

run_dir: "mistral_models/mistral-7b-instruct-v0.3_trained"  # Fill

wandb:
  project: None # your wandb project name
  run_name: "" # your wandb run name
  key: "" # your wandb api key
  offline: True

"""

# save the same file locally into the example.yaml file
import yaml
with open('example.yaml', 'w') as file:
    yaml.dump(yaml.safe_load(config), file)


In [30]:
# make sure the run_dir has not been created before
# only run this when you ran torchrun previously and created the /content/test_ultra file
# ! rm -r /content/test_ultra

import os
os.environ["WANDB_MODE"] = "disabled"


In [31]:
# start training
!rm -rf /content/mistral-finetune/mistral_models/mistral-7b-instruct-v0.3_trained

!torchrun -m train example.yaml

2025-03-10 12:17:16.401256: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-10 12:17:16.419349: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741609036.441052   25394 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741609036.447691   25394 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-10 12:17:16.469664: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [19]:
!zip -r /content/mistral-finetune/mistral_models/mistral-7b-instruct-v0.3_trained.zip /content/mistral-finetune/mistral_models/mistral-7b-instruct-v0.3_trained



  adding: content/mistral-finetune/mistral_models/mistral-7b-instruct-v0.3_trained/ (stored 0%)
  adding: content/mistral-finetune/mistral_models/mistral-7b-instruct-v0.3_trained/checkpoints/ (stored 0%)
  adding: content/mistral-finetune/mistral_models/mistral-7b-instruct-v0.3_trained/checkpoints/checkpoint_000300/ (stored 0%)
  adding: content/mistral-finetune/mistral_models/mistral-7b-instruct-v0.3_trained/checkpoints/checkpoint_000300/consolidated/ (stored 0%)
  adding: content/mistral-finetune/mistral_models/mistral-7b-instruct-v0.3_trained/checkpoints/checkpoint_000300/consolidated/tokenizer.model.v3 (deflated 61%)
  adding: content/mistral-finetune/mistral_models/mistral-7b-instruct-v0.3_trained/checkpoints/checkpoint_000300/consolidated/lora.safetensors (deflated 21%)
  adding: content/mistral-finetune/mistral_models/mistral-7b-instruct-v0.3_trained/checkpoints/checkpoint_000300/consolidated/params.json (deflated 49%)
  adding: content/mistral-finetune/mistral_models/mistral-7b

## Inference

In [20]:
!pip install mistral_inference

In [25]:
from mistral_inference.transformer import Transformer
from mistral_inference.generate import generate

from mistral_common.tokens.tokenizers.mistral import MistralTokenizer
from mistral_common.protocol.instruct.messages import UserMessage
from mistral_common.protocol.instruct.request import ChatCompletionRequest

model_dir = "/content/mistral-finetune/mistral_models/mistral-7b-instruct-v0.3_trained/checkpoints/checkpoint_000300"
tokenizer = MistralTokenizer.from_file(f"{model_dir}/consolidated/tokenizer.model.v3")  # change to extracted tokenizer file
model = Transformer.from_folder(f"/content/mistral_models")  # change to extracted model dir
model.load_lora(f"{model_dir}/consolidated/lora.safetensors")

completion_request = ChatCompletionRequest(messages=[UserMessage(content="Create a full workflow JSON action for this instruction: Filter the 'temperatures' table to include only values greater than 36.")])

tokens = tokenizer.encode_chat_completion(completion_request).tokens

out_tokens, _ = generate([tokens], model, max_tokens=64, temperature=0.0, eos_id=tokenizer.instruct_tokenizer.tokenizer.eos_id)
result = tokenizer.instruct_tokenizer.tokenizer.decode(out_tokens[0])

import json
r = json.loads(result)
print(json.dumps(r, indent=2))

JSONDecodeError: Unterminated string starting at: line 1 column 208 (char 207)